In [1]:
%matplotlib inline
import matplotlib
import seaborn as sns
matplotlib.rcParams['savefig.dpi'] = 144

In [2]:
import pandas as pd

# SQL - Structured Query Language
<!-- requirement: small_data/2013_Gaz_tracts_national.tsv -->

SQL is a formal language for communicating with a database to articulate precisely what information you want from a collection of database tables.

It is possibly the most commonly used language in the world (currently the most requested language in job descriptions according to [Code-dojo](http://www.codingdojo.com/blog/9-most-in-demand-programming-languages-of-2017/)!).  You **will** be asked about it in a job interview if you're applying to be a Data Scientist or a Data Analyst.

Databases are *structured* repositories for data in that database designers must be specific about defining a common set of data items that will be available for each record in a table. These have * **typed** columns*, and data entries are in *rows*.

Python has a built-in tools for accessing a database and sending or receiving data. In this notebook, we will explore these tools.

To begin, consider the abstract concept of a table.

Imagine you have this collection of information about a list of your friends.

 index |   name   |     city      | favorite_color 
:-----:|:--------:|:-------------:|:--------------:
   0   |   John   |   New York    |      blue      
   1   |  Jason   |    Chicago    |     yellow     
   2   |   Mary   | Washington DC |      blue      
   3   |  Jason   |    Dallas     |      cyan      
   4   |  Susan   |    Seattle    |     green      
   5   | Beatrice |    Chicago    |     violet     

For each friend, we list their `name`, the `city` they live in, and their `favorite_color`. The data is organized into rows and columns. Each row represents a different friend, and each column represents similar data we know about each friend.

There's also an extra `index` column thrown in. The purpose of this column is to provide a unique identifier for each row. In this case it is an integer, kind of like a row number, but this need not be the case. It is helpful (but not required) to have at least one column in a table that is unique. This allows us to uniquely identify a particular row. For this table such a thing is important because we have two friends named "Jason", two friends who live in Chicago, and two friends who like the color blue.

Now imagine you want to plot where your friends live on a map. To do this you need to know your friends' names and cities. You don't need to know their favorite colors right now, and the index is not important. You just need to **SELECT** the `name` and `city` columns:

   name   |     city      
:--------:|:-------------:
   John   |   New York    
  Jason   |    Chicago    
   Mary   | Washington DC 
  Jason   |    Dallas     
  Susan   |    Seattle    
 Beatrice |    Chicago    

Now imagine you are visiting Chicago next week and you want to recall information about which of your friends who live there. You want to look at the rows **WHERE** `city = 'Chicago'` and exclude the others:

   name   |  city   | favorite_color 
:--------:|:-------:|:--------------:
  Jason   | Chicago |     yellow     
 Beatrice | Chicago |     violet     

When you are in Chicago you are in clothing store and want to buy each friend a sweater in their favorite color. To buy the correct sweaters, you need to **SELECT** the `name` and `favorite_color` columns for the rows **WHERE** `city = 'Chicago'`.

   name   | favorite_color 
:--------:|:--------------:
  Jason   |     yellow     
 Beatrice |     violet     

The store makes hand-knit sweaters in several colors. The store owners have their own table for the prices they wish to charge for each colored sweater:

 sweater_color | price 
:-------------:|:-----:
      red      | 59.99 
    yellow     | 49.99 
     blue      | 79.99 
    violet     | 89.99 

If we want to know how much these sweaters will cost for each friend, first we must **join** the store owner's table with our information about the favorite colors of our friends who live in Chicago. We **join** data together by selecting rows in both tables **WHERE** `favorite_color = sweater_color`:

   name   | favorite_color | price 
:--------:|:--------------:|:-----:
  Jason   |     yellow     | 49.99 
 Beatrice |     violet     | 89.99 
 
And finally, to get the total cost, we must **SUM** the prices in the `price` column after joining the two tables together. We find the total cost is $139.98.

## Friends example in SQL

We can repeat our previous example in Python using sqlite. Jupyter notebooks come with a SQL extension that we will  use to connect to a test database we will create for this occasion.

In [21]:
%load_ext sql
%sql sqlite:///testdb.sqlite

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


u'Connected: None@testdb.sqlite'

First we must **create** a table to store information about our friends. We also need a table for the store's sweater price data.

In [22]:
%%sql
--#This makes a SQL cell - everything in here will be interpreted as SQL

--#I'm a comment.  My # isn't necessary, but makes the colors display correctly in Jupyter

--#Newlines are ignored, space them to make it more readable
--#Note that the where criteria *do not* need to be part of the selected columns

--#Generally not a good idea in real databases, deletes a table
DROP TABLE IF EXISTS friends;

--#Table creation - must specify a type for each column
--#if NOT NULL is not set, entries can be blank (NULL)
CREATE TABLE friends (
    idx               INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL,
    name              TEXT NOT NULL,
    city              TEXT NOT NULL,
    favorite_color    TEXT
);

DROP TABLE IF EXISTS merchandise;

CREATE TABLE IF NOT EXISTS merchandise (
    sweater_color     TEXT NOT NULL,
    price             NUMBER NOT NULL
);

Done.
Done.
Done.
Done.


[]

Our tables are empty. Let's **insert** data.

In [23]:
%%sql

INSERT INTO friends (name, city, favorite_color)
    VALUES ('John', 'New York', 'blue');

INSERT INTO friends (name, city, favorite_color)
    VALUES ('Jason', 'Chicago', 'yellow');

INSERT INTO friends (name, city, favorite_color)
    VALUES ('Mary', 'Washington DC', 'blue');

INSERT INTO friends (name, city, favorite_color)
    VALUES ('Jason', 'Dallas', 'cyan');

INSERT INTO friends (name, city, favorite_color)
    VALUES ('Susan', 'Seattle', 'green');

INSERT INTO friends (name, city, favorite_color)
    VALUES ('Beatrice', 'Chicago', 'violet');

INSERT INTO merchandise (sweater_color, price)
    VALUES ('red', 59.99), ('yellow', 49.99), ('blue', 79.99), ('violet', 89.99);


1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
4 rows affected.


[]

We can confirm that our data was inserted correctly by **SELECT**ing the data from each table.

In [24]:
%%sql

SELECT * FROM friends;

Done.


idx,name,city,favorite_color
1,John,New York,blue
2,Jason,Chicago,yellow
3,Mary,Washington DC,blue
4,Jason,Dallas,cyan
5,Susan,Seattle,green
6,Beatrice,Chicago,violet


In [25]:
%%sql

SELECT * FROM merchandise;

Done.


sweater_color,price
red,59.99
yellow,49.99
blue,79.99
violet,89.99


If we want to know our friend's names and their cities, we can select just those two columns from the friends table.

In [26]:
%%sql

SELECT name, city FROM friends;

Done.


name,city
John,New York
Jason,Chicago
Mary,Washington DC
Jason,Dallas
Susan,Seattle
Beatrice,Chicago


To get information about our friends who live in city, we can select just those rows.

In [27]:
%%sql

SELECT name, city, favorite_color FROM friends WHERE city = 'Chicago';

Done.


name,city,favorite_color
Jason,Chicago,yellow
Beatrice,Chicago,violet


WHERE's can be arbitrarily complex, with AND, OR, parenthesis, etc.  Basic logic is available like <, >, =, etc

In [28]:
%%sql

SELECT * FROM merchandise WHERE price > 50.00 AND price < 80.00;

Done.


sweater_color,price
red,59.99
blue,79.99


As are selectors such as IN and LIKE (for wildcarding)

In [29]:
%%sql

SELECT name, city FROM friends WHERE name like 'J%';

Done.


name,city
John,New York
Jason,Chicago
Jason,Dallas


In [30]:
%%sql

SELECT favorite_color, name FROM friends WHERE city IN ('Chicago','Dallas', 'New York');

Done.


favorite_color,name
blue,John
yellow,Jason
cyan,Jason
violet,Beatrice


For testing, LIMIT lets you only display the first few rows of a table

In [31]:
%%sql

SELECT favorite_color, name FROM friends 
WHERE city IN ('Chicago','Dallas', 'New York') LIMIT 3;

Done.


favorite_color,name
blue,John
yellow,Jason
cyan,Jason


To get the price of the sweater we will give to each friend, we can join our data with the merchandise data.

In [32]:
%%sql

SELECT
    f.name,
    f.favorite_color,
    m.price
FROM
    friends as f,
    merchandise as m
WHERE
    f.city = 'Chicago' AND
    f.favorite_color = m.sweater_color;

Done.


name,favorite_color,price
Jason,yellow,49.99
Beatrice,violet,89.99


Finally, to get the total cost of the sweaters:

In [33]:
%%sql

SELECT
    SUM(m.price) as total_cost
FROM
    friends as f,
    merchandise as m
WHERE
    f.city = 'Chicago' AND
    f.favorite_color = m.sweater_color;

Done.


total_cost
139.98


# SQL Syntax in Detail

**Key nouns:**
    - "Relational" database
    - Table
    - Row
    - Column
    - (Primary key)

**Key verbs:**
    - `CREATE TABLE` / `DROP TABLE` to modify the structure of the database
    - `SELECT` to read out some rows
    - `INSERT` / `UPDATE` / `DELETE` to modify (or delete) some rows of tables

**Key modifiers:**
    - `WHERE`: impose condition
    - `ORDER BY`: sort
    - `GROUP BY`: aggregate (e.g. pivot tables in Excel)
    - `JOIN`: combining tables in one query

## Creating a Table

You create a table using the syntax

```SQL
CREATE TABLE tb ([schema])
```

We can insert rows by using the syntax

```SQL
INSERT INTO tb ([column]) VALUES ([values]);
```

**Gotchas:**
 - For strings you 'must' use single quotes.  (SQLite and MySQL don't care.  Postgres does.)
 - The primary key is required to uniquely specify a row.  If you do not specify it when you insert an item (and you usually don't) a new unique key will be assigned to you.  If you try to insert a primary key value that already exists for the table, you will get an error.
 - While not necessarily required, it's a good idea to preceded table creation by table deletion:

```SQL
DROP TABLE IF EXISTS tb
```
 
Below are some examples: we'll worry about the details later.

In [ ]:
%%sql
-- This table demonstrates how to create a table, insert values, and display results
    
DROP TABLE IF EXISTS Employees1;
CREATE TABLE Employees1 (
    employee_id INTEGER PRIMARY KEY,
    name TEXT,
    age FLOAT
);

INSERT INTO Employees1 (name, age) VALUES ('Bob', 2.);
INSERT INTO Employees1 (age, name) VALUES (1., 'John');
INSERT INTO Employees1 (employee_id, age, name) VALUES (4, 3., 'Jack');

SELECT * FROM Employees1;

## Schemas
SQL is a typed language (if it can be called a language).  That means that its tables have a **schema** or a description of the structure of a database (tables, rows, etc.).  Each column of a table must specify its **type** and a few other attributes.  The basic types are:

- `INTEGER` (or `INT`): integer-valued numeric data
- `FLOAT`: non-integer numeric data
- `TEXT`: strings or textual data
- `DATE`: a date object

Columns can have optional attributes:

- `NOT NULL`: column values can always be null (not specified).  `NOT NULL` forces the table to be specified.
- `DEFAULT`: specify the default value in this column.

In [ ]:
%%sql
-- This table demonstrates optional values
    
DROP TABLE IF EXISTS Employees1;
CREATE TABLE Employees1 (
    employee_id INTEGER PRIMARY KEY,
    name TEXT NOT NULL,
    age FLOAT
);

INSERT INTO Employees1 (name) VALUES ('Bob');
INSERT INTO Employees1 (name, age) VALUES ('Jack', 3.14);

SELECT * FROM Employees1;

In [ ]:
%%sql
-- This table demonstrates non-optional values
    
DROP TABLE IF EXISTS Employees2;
CREATE TABLE Employees2 (
    employee_id INTEGER PRIMARY KEY,
    name TEXT NOT NULL,
    age FLOAT DEFAULT -1.
);

INSERT INTO Employees2 (name) VALUES ('Bob');
INSERT INTO Employees2 (name, age) VALUES ('Jack', 3.14);

SELECT * from Employees2;

## Multiple Tables and the Relational Data Model

The database consists of:
 - **Tables**: Employees, Tasks, and Assignments.  
      > Each table represents a different kind of "relation" between bits of data.
    
 - Each of the tables has many **rows** (sometimes called "relations").  For instance, each row of the `Employees` table represents information about a single employee:  
 
      > Elements of a single column are _homogeneous_ (of the same kind, or data type) but apply to _different logical entities_.
      
 - The information about a single employee is spread amongst the **columns** of the table:
 
      > Elements of a single row are _heterogeneous_ but apply to _a single logical entity_.
      
 - It is good practice to be able to specify each row of each table by a **primary key**: that is by some column, or small combination of columns, that uniquely specifies the row.  The columns `employee_id`, `task_id`, and `assignment_id` are all of this type.  This way, if you want to refer to this row in another table you can do it succinctly by this primary key.  SQL enforces uniqueness of the primary key.
      
**Examples:** here are some examples of "relations" (rows) that occur in practice:
1. Storing one-to-one pieces of data for some sort of object: i.e. for each employee we store one name and one phone number; for each project we store one deadline and one description
1. Storing one-to-many or many-to-many bits of data: i.e. if we needed to be able to store _many_ phone numbers for each employee, we'd have a new table `Employee_Phone_Numbers` with columns `employee_id` and `phone_number` (the two together can be a _composite primary key_).
    
**Anti-examples:** alternate, but **worse**, ways to do it would be: 

1. Adding columns to the `Employees` table for `phone_number_1` to `phone_number_10`.  This is bad because it makes it hard to query against the phone numbers and is inflexible if you want to change the number of phone numbers.  In the _object oriented_ or _key-value_ you could simply make phone_numbers a list or array, and all would be well -- this is not available in the relational model. 
1. Having the `Employee_Phone_Numbers` table duplicate the employee name and other information.  This is bad because duplication of information across tables wastes space and can lead to update errors.
   

In [ ]:
%%sql
DROP TABLE IF EXISTS Employees;
CREATE TABLE Employees (
    employee_id INTEGER PRIMARY KEY,
    name TEXT NOT NULL,
    hire_date DATE NOT NULL,
    title TEXT NOT NULL,
    phone_number TEXT NOT NULL
);

INSERT INTO Employees VALUES (1, 'Jenny Smith', '2010-12-1', 'Chief Music Office', '867-5309');
INSERT INTO Employees (title, name, phone_number, hire_date) VALUES ('CEO', 'John Doe', '555-1234', '2011-12-1');
INSERT INTO Employees (title, name, phone_number, hire_date) VALUES ('Worker', 'Worker Bee 1', '555-1201', '2012-12-01');
INSERT INTO Employees (title, name, phone_number, hire_date) VALUES ('Worker', 'Worker Bee 2', '555-1202', '2012-12-03');
INSERT INTO Employees (title, name, phone_number, hire_date) VALUES ('Worker', 'Worker Bee 3', '555-1203', '2012-12-05');
INSERT INTO Employees (title, name, phone_number, hire_date) VALUES ('Worker', 'Worker Bee 4', '555-1204', '2012-12-07');

SELECT * FROM Employees;

In [ ]:
%%sql
DROP TABLE IF EXISTS Tasks;
CREATE TABLE Tasks (
    task_id INTEGER PRIMARY KEY,
    deadline DATE NOT NULL,
    status INT NOT NULL DEFAULT 0,
    description TEXT
);

INSERT INTO Tasks (deadline, description, status) VALUES ('2012-12-11', 'Hire worker bees.', 1);
INSERT INTO Tasks (deadline, description) VALUES ('2014-07-18', 'Save the world.');
INSERT INTO Tasks (deadline, description, status) VALUES ('2014-12-11', 'Cost cutting on worker bees.', 0);

SELECT * FROM Tasks;

In [ ]:
%%sql
DROP TABLE IF EXISTS Assignments;
CREATE TABLE Assignments (
    assignment_id INTEGER PRIMARY KEY,
    task_id INTEGER NOT NULL,
    employee_id INTEGER NOT NULL,
    start_date DATE NOT NULL,
    end_date DATE NOT NULL,
    charge_code TEXT NOT NULL,  -- This should probably be its own table!
    fraction_assignment REAL NOT NULL
);

INSERT INTO Assignments (task_id, employee_id, start_date, end_date, fraction_assignment, charge_code) VALUES (1, 2, '2012-01-01', '2012-12-31', 0.5, 'INTERNAL');
INSERT INTO Assignments (task_id, employee_id, start_date, end_date, fraction_assignment, charge_code) VALUES (2, 3, '2013-01-01', '2014-07-18', 1.0, 'CJX5');
INSERT INTO Assignments (task_id, employee_id, start_date, end_date, fraction_assignment, charge_code) VALUES (2, 4, '2013-01-01', '2014-07-18', 1.0, 'CJX5');
INSERT INTO Assignments (task_id, employee_id, start_date, end_date, fraction_assignment, charge_code) VALUES (2, 5, '2013-01-01', '2014-07-18', 1.0, 'CJX5');
INSERT INTO Assignments (task_id, employee_id, start_date, end_date, fraction_assignment, charge_code) VALUES (2, 6, '2013-01-01', '2014-07-18', 1.0, 'CJX5');
INSERT INTO Assignments (task_id, employee_id, start_date, end_date, fraction_assignment, charge_code) VALUES (2, 2, '2013-01-01', '2014-07-18', 0.75, 'CJX5');
INSERT INTO Assignments (task_id, employee_id, start_date, end_date, fraction_assignment, charge_code) VALUES (3, 2, '2013-01-01', '2014-07-18', 0.25, 'INTERNAL');

SELECT * FROM Assignments;

In [ ]:
%%sql

DROP TABLE IF EXISTS Pension_Choices;
CREATE TABLE Pension_Choices (
    pension_id INTEGER PRIMARY KEY,
    employee_id INTEGER NOT NULL, 
    plan_name TEXT NOT NULL,
    funding_amount INTEGER NOT NULL);

INSERT INTO Pension_Choices (employee_id, plan_name, funding_amount) VALUES (3, 'Retirement Plan A', '3000');
INSERT INTO Pension_Choices (employee_id, plan_name, funding_amount) VALUES (4, 'Retirement Plan A', '5000');
INSERT INTO Pension_Choices (employee_id, plan_name, funding_amount) VALUES (5, 'Retirement Plan B', '5000');
INSERT INTO Pension_Choices (employee_id, plan_name, funding_amount) VALUES (6, 'Retirement Plan B', '1000');

SELECT * FROM Pension_Choices;

## Selecting data
The basic way to read rows from a single table is

```SQL
SELECT expression1, expression2, ... FROM table_name [WHERE condition] [GROUP BY columns] [ORDER BY columns] [LIMIT number];
```

Brackets denote parts which are optional.  We'll explain everything except for `GROUP BY`.  Let us explain the parts:
   1. __"Expressions": __
   
   The basic example is just a column name.  Another example is the placeholder `*`: it indicates _all_ columns.  You can also perform arithmetic (e.g. `column_1 + column_2`) and similar minor operations.

   1. __`WHERE` ("condition"):__
   
   An example would be `column_name = value`, and for partial string matches there is a special operator `LIKE`.  You can combine these with the usual binary operators (AND, OR) and parentheses for grouping.

   1. __`ORDER BY` (sorting):__
   
   You can add a `SORT BY column_name1, column_name2, ...` to sort the results by the given column (applied in left-to-right order).  By default the sorting is ascending.  If you want it to be descending write e.g. `column_name1 DESC` (you can also write `ASC` for ascending to disambiguate).
   
   1. __`GROUP BY`:__
   
   Aggregation control - sets the column(s) that are used to control which elements become the new, unique "keys" of the new table
 
   1. __`LIMIT`:__
   
   If you want to just get at most the first N rows that would be returned, add on `LIMIT N`.
   
   1. __`AS` (named expressions):__
   
   You can specify new names for the expressions that you're selecting.  This is especially useful when you're using aggregate functions: 
```SQL
SELECT SUM(salary) as 'Total Salary' FROM ...
```
  
Finally two remarks: SQL is not case sensitive.  It is just common to use all-caps for the commands, and normal case for table / column names.  It is also whitespace and new-line insensitive, so you can format your commands on multiple lines as you please.


Now for a bunch of examples:

In [ ]:
%%sql

SELECT
    count(*) as Count
FROM
    Tasks
WHERE
    status=0;

In [ ]:
%%sql

SELECT
    assignment_id,
    julianday(end_date) - julianday(start_date) AS duration
FROM
    Assignments
WHERE
    employee_id=2;

## CASE WHEN

Another useful structure that can be an alternative to `WHERE` statements is the `CASE WHEN` command. It essentially operates as a switch statement where you can specify multiple conditions and different outcomes for each condition.

In [ ]:
%%sql

SELECT
    assignment_id, CASE
        WHEN julianday(end_date) - julianday(start_date) < 365 THEN "months"
        WHEN julianday(end_date) - julianday(start_date) > 365 THEN "years"
        ELSE "one year"
        END "length"
FROM Assignments;

## Groupby (aggregation)
   
Finally, dd a `GROUP BY column_name1, ...` term to aggregate the results along the listed columns.   This means that for _other_ columns you mention in the expressions you must explicitly specify an _aggregate method_ (e.g. `SUM(column_1)`).
   
   You can use these aggregate methods even without a GROUP BY, to get information for the whole table: e.g.
  ```SQL 
SELECT COUNT(1), SUM(salary) FROM employees GROUP BY department;
   ```
   returns the number of employees and their total salary by department.  Below are some more examples

In [ ]:
%%sql

SELECT plan_name, count(1), sum(funding_amount) FROM Pension_choices GROUP BY plan_name

In [ ]:
%%sql

SELECT
    task_id,
    SUM((julianday(end_date) - julianday(start_date))*fraction_assignment) as Total_Worker_Days
FROM
    Assignments
GROUP BY task_id
ORDER BY Total_Worker_Days DESC;

## Combining tables: Joins and subqueries

It's pretty boring to only use one table.  We can combine data from multiple tables in two main ways: joins and subqueries.

**Nicknames:**

There's a common bit of syntax involved in each: you can write

    table_name name
    
or
    
    table_name AS name

to say that table `table_name` is going to participate in this query with a "nickname" of name.  Then, to refer to one of its columns you write

    name.column_name

You could also have not given it the nickname, and then you could've written `table_name.column_name`... but that gets old fast.  Also, in some fancy examples the same table enters into a query twice -- at which point you need to name them.


**Subqueries:**
It turns out that both the expressions you were SELECTing __and__ the "table" you were selecting FROM can be subqueries (__and__ quantities in WHEREs)!  Let's just see it by example:

In [4]:
%%sql

SELECT a.task_id, 
    (SELECT description 
     FROM Tasks t WHERE t.task_id=a.task_id) as description, 
    SUM((julianday(a.end_date) - julianday(a.start_date)) 
                * a.fraction_assignment) as Total_Worker_Days
    FROM Assignments a
    GROUP BY a.task_id
    ORDER BY Total_Worker_Days DESC;

(sqlite3.OperationalError) no such table: Assignments [SQL: u'SELECT a.task_id, \n    (SELECT description \n     FROM Tasks t WHERE t.task_id=a.task_id) as description, \n    SUM((julianday(a.end_date) - julianday(a.start_date)) \n                * a.fraction_assignment) as Total_Worker_Days\n    FROM Assignments a\n    GROUP BY a.task_id\n    ORDER BY Total_Worker_Days DESC;']


In [ ]:
%%sql

SELECT ad.task_id, ad.Total_Worker_Days 
    FROM (SELECT task_id, SUM((julianday(end_date) - julianday(start_date))*fraction_assignment) as Total_Worker_Days
            FROM Assignments
            GROUP BY task_id
            ORDER BY Total_Worker_Days DESC) as ad
    WHERE ad.task_id=(SELECT task_id FROM Tasks WHERE description LIKE 'Save%');

**Joins:**
Many things that can be expressed by a subquery can be expressed more clearly (and will often run faster) as a `join`.   

The basic idea of joins is to imagine taking the _product_ of your two tables: i.e. a row is now an ordered pair of a row of table 1 and a row of table 2.  You could -- of course -- do this and make it its own table, but the ability to write JOINs means that you don't have to waste the storage space and risk un-needed duplication of data.

The join in the previous paragraph is called the full or outer join, because there are no conditions.  In practice, you'll want to take the subset of the product where some condition holds (i.e. "join" the two tables along a common id column).  This can be expressed pretty much exactly as I just said, in SQL:
    
```SQL
       SELECT * FROM table1, table2
                WHERE table1.column_name1 = table2.column_name2;
```

In SQL terms, this is called an `INNER JOIN`: You do not get a row in the result of the SELECT unless you can match a row in table 1 with a match in table 2 along the given columns.  There is alternate syntax for it:
```SQL
SELECT * FROM table1
        INNER JOIN table2
        ON table1.column_name1=table2.column_name2;
```

In [ ]:
%%sql

SELECT a.*, t.* 
  FROM Assignments a, Tasks t 
  WHERE a.task_id=t.task_id 
    AND employee_id=2 
    AND status=0
  ORDER BY t.deadline;

## UNION and UNION ALL

These commands are used for joining data row-wise either between tables or within the same table. In general, the syntax looks like `(SELECT statement) UNION (SELECT STATEMENT)` where both SELECT statements have the same columns.

You can imagine that this can be used for both filtering and aggregation of records.

The difference between UNION and UNION ALL is that the former removes duplicate records (and therefore is more expensive performance-wise) and the latter does not.

In [ ]:
%%sql

SELECT task_id, employee_id FROM Assignments WHERE employee_id = 2
UNION ALL
SELECT task_id, employee_id FROM Assignments WHERE employee_id = 4;

### Further reading

SQL:
- http://www.w3schools.com/sql/default.asp (and http://www.w3schools.com/sql/sql_quickref.asp)

Relational databases and the normal forms:
- http://en.wikipedia.org/wiki/Database_normalization#Normal_forms

## Modifying tables

There are also queries of the form

        DELETE FROM table_name [WHERE condition]
        
        UPDATE table_name SET column_name1=value [WHERE condition];

that delete / modify rows as you might expect.  You can combine both of these with inner queries (and in Postgres, you can combine UPDATE with JOINs) for interesting effects.

To modify columns there are commands of the form

        ALTER TABLE table_name DROP COLUMN column_name;        
        ALTER TABLE table_name ADD COLUMN column_name column_type;
        
for instance

        ALTER TABLE Employees DROP COLUMN phone_number;
        ALTER TABLE Employees ADD COLUMN salary INT;

## More about SQLite

If you want to use SQL and don't have too much data (less than a few gigs at a time), you can use `sqlite3`.  It's already installed on DigitalOcean, is easy to use, and has excellent (and lenient) support for importing and exporting CSV.

The following commands do not work through the Jupyter notebook. You will need to open a terminal and run this command to start the interactive sqlite application.

    $sqlite3 testdb.sqlite

You will then be presented the sqlite command prompt.

### Importing CSV
To import tables to CSV:
>    
    .mode csv
    .import tasks.csv Tasks
    .import employees.csv Employees
    .import assignments.csv Assignments

Note that we didn't have to specify the schema ahead of time (though we might want to).  SQLite will take column names from the CSV header line, and will make all fields have type TEXT and be NULLable.

### Exporting to CSV
And to export:
>           
    .mode csv
    .headers on
    .output tasks.csv
    SELECT * FROM Tasks;
    .output employees.csv
    SELECT * FROM Employees;
    .output assignments.csv
    SELECT * FROM Assignments;        

### Listing tables / table schema
>      
    .tables
    .schema Tasks

## Creating SQL Tables with Pandas: A Shortcut

Pandas is awesome in many ways, and one of them is its ability to create database tables for you. If you have a DataFrame and a database connection, it will write the SQL CREATE statements to build the table schema and insert all of the rows. It works like this.

In [ ]:
import sqlite3

# sample data
fha_by_tract = pd.read_csv('small_data/2013_Gaz_tracts_national.tsv', sep='\t')
fha_by_tract.columns = [c.strip() for c in fha_by_tract.columns]

# create db connection
conn = sqlite3.connect('testdb.sqlite')

# create a database table with the proper types to match the DataFrame types and insert all of the rows.
fha_by_tract.to_sql('gaz_tracts', conn, if_exists='replace', index=False)

That's it. If you have a DataFrame and like it, you can make a database table out of it.

Now we can verify the table exists and has data.

In [ ]:
%%sql

SELECT * FROM gaz_tracts LIMIT 10;

You can also retrieve the data using the `sqlite3` connection we created. Rows are retrieved as tuples.

In [ ]:
cursor = conn.cursor()

cursor.execute("SELECT * FROM gaz_tracts LIMIT 10")

for row in cursor.fetchall():
    print row

## Another choice: Postgresql

### Quick facts

**Running a server:**
- On your DO droplets, once you install postgres it will automatically set up the server to run in the background on startup.  The actual database files are hidden somewhere inside of `/var`.

- Alternatively, you can run a "local" postgres server whose data lies in a specific directory and on a specific port:

>         initdb dir/
>         postgres [-p 8765] -D dir/

(You may want to specify a custom port that is not my made up 8765 -- the default is __5432__.)

**Quick start guide:**
After the above is done and you're running a postgres server locally, just do

         createdb [-p 8765] my_db_name

to create a new database.  When you're done with it

        dropdb --if-exists [-p 8765] my_db_name

To get a command-line console for running commands against your database, just run

        psql [-p 8765] my_db_name

(If it's running on the standard port, you don't need to specify the `-p 8765`.)

By default, this will all use a user named for your Unix username -- for the system Postgres install, this user ("role" in Postgres terms) was created when setting up your droplet.

**Postgres-specific basics:**
To list all tables or find out info about a table do:

        \d
        \d table_name

**Importing (CSV) data:**
If your data is pretty clean, the `\COPY` special command should do it

        \COPY table_name FROM 'path/to/my/file.csv' WITH CSV DELIMITER ',' NULL AS '';
        
**Exporting (CSV) data:** You can use `\COPY` in both directions
        
        \COPY table_name TO 'path/to/my/file.csv' WITH DELIMITER ',' CSV HEADER;

## SQL integration in Python

The most common usage mode for you will be importing / exporting to CSV, or working interactively through `sqlite3` / `psql`.

You can also interact between Python and SQL.  We've been doing it throughout this whole notebook using the `%sql` magic.  There are also libraries for doing it less magically

 - `psycopg2` for communicating with Postgres
 - `sqlalchemy` for communicating with most anything, and some fancy ORM tricks

### What to use?
-------------

**SQLite:**
-------------
_In short:_ Use this if your data is smallish (several GB max) and you want to do some SQL-based processing, _and_ if you know ahead of time that you will not need to do operations in parallel. Probably the preferred choice for mini-projects.

**Pros**
 - It operates on a single flat file on disk, which you can treat like any other file -- copy, scp, rename, etc.
 - It is a single small executable, comes pre-installed on many things, and doesn't require running a server.
 - Extensible (by C code).  It's open source and is small enough to be easy to modify it / write extension modules in C.
 - It has a quick and permissive (of errors) CSV loader, and doesn't stress correctness of schemas.

**Cons**
 - It lacks some useful "advanced" features (in particular, JOINs in UPDATEs have to be faked via subqueries and/or temporary tables).  The workarounds are generally slower than the "right" approach would be.
 - It's not multi-user.  If you're running a long-running computation, you can't do anything else with it.

**Postgresql:**
----------------
_In short:_  The premier free "real" RDBMS (i.e. SQL-type database).  Use it if you need SQL semantics and have more than several GB of data or need multi-user / multi-process access.  
[Exception: If you have lots of data and do lots of easily parallelizable tasks, and will be running either on a cluster on a machine that fits your data in RAM, you may be better off with a specialized SQL-like system: e.g. Hive, Shark, MemSQL (commercial).]

**Pros**
 - Excellent support of SQL features, good performance and `"EXPLAIN ..."` for seeing query plans and tweaking performance.
 - Several nice SQL extensions: JSON-column support, and `Postgis` for GIS-based geographical data types and queries.
 - Extensible (in a variety of languages, _including Python_).
 
**Cons**
 - You have to run a server.
 - The CSV loader is quick on correct data, but simply doesn't accept malformed data.  Does stress correctness of schemas.

 **Other options:**
 ------------------
 - *If your data is small enough to fit _comfortably_ into memory:* 
 
   You can get by with something like `pandas` which allows for much quicker visualization and has better built in statistical support.  (There is also support for running SQL queries against pandas.  It uses sqlite behind the scenes -- beware that doing this has a performance cost over just using sqlite originally.)
 - *If you have have *lots* of data (i.e. more than any one machine you have can store on disk) or if you want to perform a task that calls for cross-machine parallelization (e.g. it makes sense to run on Hadoop):*
 
   There are specialized SQL-like databases made for this, notably `Hive` which is built on top of Hadoop.  (Also: `Shark` and `Impala` which do the same thing with different size and performance characteristics.)
 - *If your data set is large, but you happen to have a _very_ beefy sever (or a cluster of them) with enough RAM to fit it all:*
 
  There are SQL-like databases optimized for in-memory use, including `MemSQL` and also modes of `Shark` and `Impala` mentioned above.

### Exercises

1. Find a CSV file somewhere on the Internet that contains data that interests you. Put the data into a table in sqlite.
1. Query the data in a meaningful way and comment on the results.

### Exit Tickets

1. What is a Database?
1. What are the differences and similarities between sqlite and postgresql?
1. What are the keywords in SQL syntax? Which keywords do you expect to use the most often?

*Copyright &copy; 2016 The Data Incubator.  All rights reserved.*